# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
# for root, dirs, files in os.walk(filepath):
#     print('-------')
#     print(root, dirs, files)
    
# # join the file path and roots with the subdirectories using glob
#     file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

# NOTE: Commented out above section because it incorrectly collects any csv temp
# files created by jupyter lab when you open one of the csv files (they're in 
# ./event_data/.ipynb_checkpoints/*.csv.  Globbed by taking files from event_data
# directly
file_path_list = glob.glob(os.path.join(filepath, '*'))
for f in file_path_list:
    print(f)

/home/workspace
/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/event_data/2018-11-25-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/hom

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


(does not match total lines because it is skipping rows with blanks)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS project1b
WITH REPLICATION = 
{'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [7]:
session.set_keyspace('project1b')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
class RowData:
    def __init__(self, line):
        """
        Parser for event_data files
        """
        self.line = line
        self.artist = self.line[0]
        self.firstName = self.line[1]
        self.gender = self.line[2]
        self.itemInSession = int(self.line[3])
        self.lastName = self.line[4]
        self.length = float(self.line[5])
        self.level = self.line[6]
        self.location = self.line[7]
        self.sessionId = int(self.line[8])
        self.song = self.line[9]
        self.userId = int(self.line[10])

**1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

The primary key for this table was chosen to support a WHERE clause on sessionId and itemInSession
* sessionId has been used as the partition key here because it is expected to have more unique values (for better data distribution flexibility).  This also supports a future query of *all* songs from a session, if desired
* Based on the provided sample data, (sessionId, itemInSession) provides a unique key for each row.  If this is not true, the additional clustering columns (or some other columns to provide uniqueness) would be required

In [9]:
drop_artist_song = "DROP TABLE IF EXISTS artist_song"
session.execute(drop_artist_song)

In [10]:
create_artist_song = """CREATE TABLE IF NOT EXISTS artist_song (
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    length FLOAT,
    PRIMARY KEY (sessionId, itemInSession)
)"""

session.execute(create_artist_song)

In [11]:
insert_artist_song = "INSERT INTO artist_song (sessionId, itemInSession, artist, song, length) "
insert_artist_song += "VALUES (%s, %s, %s, %s, %s)"

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        line = RowData(line)
        session.execute(insert_artist_song, (line.sessionId, line.itemInSession, line.artist, line.song, line.length))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
query_artist_song = "SELECT artist, song, length FROM artist_song WHERE sessionId=%s AND itemInSession=%s"

In [14]:
results = session.execute(query_artist_song, (338, 4))

for r in results[:5]:
    print(r)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


**2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

The primary key for this table was chosen to support a WHERE clause on userId and sessionId
* (userId, sessionId) has been used as the partition key here to allow partitioning on each user session
* Additional clustering column of itemInSession included to provided sorting of songs returned

In [15]:
drop_artist_song_user = "DROP TABLE IF EXISTS artist_song_user"
session.execute(drop_artist_song_user)

In [16]:
create_artist_song_user = """CREATE TABLE IF NOT EXISTS artist_song_user (
    userId INT,
    sessionId INT,
    itemInSession INT,
    artist TEXT,
    song TEXT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY ((userId, sessionId), itemInSession)
)"""

session.execute(create_artist_song_user)          

In [17]:
insert_artist_song_user = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
insert_artist_song_user += "VALUES (%s, %s, %s, %s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        line = RowData(line)
        session.execute(insert_artist_song_user, (line.userId, line.sessionId, line.itemInSession, line.artist, line.song, line.firstName, line.lastName))

In [18]:
query_artist_song_user = "SELECT artist, song, firstName, lastName FROM artist_song_user WHERE userId=%s AND sessionId=%s"

In [19]:
results = session.execute(query_artist_song_user, (10, 182))

for r in results[:5]:
    print(r)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


**3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

The primary key for this table was chosen to support a WHERE clause on song name.  
* song alone does not provide a sufficient primary key.  Many users will listen to the same song, thus without additional primary key columns our table will overwrite a song's row whenever a user listens the song (effectively capturing the *most recent* listener to a song, not all listeners)
* userId has been added to the primary key in order to ensure uniqueness

In [20]:
drop_user_listened_to = "DROP TABLE IF EXISTS user_listened_to"
session.execute(drop_user_listened_to)

In [21]:
create_user_listened_to = """CREATE TABLE IF NOT EXISTS user_listened_to (
    song TEXT,
    userId INT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY (song, userId)
)"""

session.execute(create_user_listened_to)          

In [22]:
insert_user_listened_to = "INSERT INTO user_listened_to (song, userId, firstName, lastName) "
insert_user_listened_to += "VALUES (%s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        line = RowData(line)
        session.execute(insert_user_listened_to, (line.song, line.userId, line.firstName, line.lastName))

In [23]:
query_user_listened_to = "SELECT firstName, lastName FROM user_listened_to WHERE song=%s"

In [24]:
results = session.execute(query_user_listened_to, ("All Hands Against His Own",))

for r in results[:5]:
    print(r)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [25]:
session.execute(drop_artist_song)
session.execute(drop_artist_song_user)
session.execute(drop_user_listened_to)

### Close the session and cluster connection¶

In [26]:
session.shutdown()
cluster.shutdown()